# cambio de pesos
aqui se hace el metodo maape, recibe 4 archivos, el de la precipitacion de la estacion, el inicio de lluvia real y los indices de pac y atrlantico

In [20]:
pwd

'c:\\Users\\djaimes\\Desktop\\icc'

In [21]:
import pandas as pd
import numpy as np

# =========================
# FUNCIONES AUXILIARES
# =========================

def calcular_maape(y_true, y_pred):
    """
    Calcula el MAAPE (Mean Arctangent Absolute Percentage Error)
    entre dos listas o arrays numéricos.
    """
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.arctan(np.abs((y_true - y_pred) / (y_true + 1e-6))))

def construir_serie(df, año, mes, var):
    """
    Construye una serie de 6 meses consecutivos terminando en abril del año actual:
    Dic y Nov del año anterior, y Ene a Abr del año actual.
    """
    df = df[['year', 'month', var]]
    meses = [(año-1, 12), (año-1, 11), (año, 1), (año, 2), (año, 3), (año, 4)]
    serie = []
    for y, m in meses:
        valor = df[(df['year'] == y) & (df['month'] == m)]
        if valor.empty:
            return None
        serie.append(valor.iloc[0][var])
    return serie

# =========================
# CARGA DE DATOS
# =========================

ruta_prec = r"c:\\Users\\djaimes\\Desktop\\icc\data\puertosanjose_prec.xlsx" #year-month-prec
#ruta_real = r"c:\\Users\\djaimes\\Desktop\\icc\\data\\start_prec_stations.xlsx"
ruta_real = r"c:\\Users\\djaimes\\Desktop\\icc\data\start_prec_puertosanjose.xlsx"#year-sem_iell
ruta_atl = "c:\\Users\\djaimes\\Desktop\\icc\\data\\oni_atlantico.xlsx"
ruta_pac = "c:\\Users\\djaimes\\Desktop\\icc\\data\\oni_pacifico.xlsx"

prec = pd.read_excel(ruta_prec)
real = pd.read_excel(ruta_real)
atl = pd.read_excel(ruta_atl)
pac = pd.read_excel(ruta_pac)

# Renombrar columnas para estandarizar nombres
atl.rename(columns={'oni': 'atl'}, inplace=True)
pac.rename(columns={'oni': 'pac'}, inplace=True)

# =========================
# CONFIGURACIÓN
# =========================

año_actual = 2019
mes_actual = 4
nombre_estacion = 'puertosanjose'

escenarios = {
    'prec_dom': {'prec': 0.7, 'pac': 0.15, 'atl': 0.15},
    'enso_dom': {'prec': 0.3, 'pac': 0.5, 'atl': 0.2},
    'enso_prec': {'prec': 0.4, 'pac': 0.4, 'atl': 0.2},
    'neutro': {'prec': 0.33, 'pac': 0.33, 'atl': 0.33},
    'atl_dom': {'prec': 0.3, 'pac': 0.2, 'atl': 0.5},
}

# =========================
# CÁLCULO DE SERIES ACTUALES
# =========================

serie_actual_prec = construir_serie(prec, año_actual, mes_actual, 'prec')
serie_actual_atl = construir_serie(atl, año_actual, mes_actual, 'atl')
serie_actual_pac = construir_serie(pac, año_actual, mes_actual, 'pac')

if None in [serie_actual_prec, serie_actual_atl, serie_actual_pac]:
    print("No hay suficientes datos para calcular la serie actual.")
    exit()

# =========================
# COMPARACIÓN CON AÑOS ANTERIORES
# =========================

resultados = []

for año_pasado in range(año_actual-1, prec['year'].min()-1, -1):
    serie_ant_prec = construir_serie(prec, año_pasado, mes_actual, 'prec')
    serie_ant_atl = construir_serie(atl, año_pasado, mes_actual, 'atl')
    serie_ant_pac = construir_serie(pac, año_pasado, mes_actual, 'pac')

    if None in [serie_ant_prec, serie_ant_atl, serie_ant_pac]:
        continue

    maape_prec = calcular_maape(serie_actual_prec, serie_ant_prec)
    maape_atl = calcular_maape(serie_actual_atl, serie_ant_atl)
    maape_pac = calcular_maape(serie_actual_pac, serie_ant_pac)

    #real = real[real.estacion==nombre_estacion]
    fila_real = real[real['year'] == año_pasado]
    semana_inicio = int(fila_real['sem_iell'].values[0]) if not fila_real.empty else np.nan

    resultados.append({
        'año': año_pasado,
        'maape_prec': maape_prec,
        'maape_atl': maape_atl,
        'maape_pac': maape_pac,
        'sem_iell': semana_inicio
    })

if len(resultados) < 8:
    print("No hay suficientes años anteriores con datos completos para comparar.")
    exit()

df_resultados = pd.DataFrame(resultados)

# =========================
# CÁLCULO DE ESCENARIOS
# =========================

df_final = pd.DataFrame()

for nombre_esc, pesos in escenarios.items():
    df_esc = df_resultados.copy()
    df_esc['maape_esc'] = (
        pesos['prec'] * df_esc['maape_prec'] +
        pesos['atl'] * df_esc['maape_atl'] +
        pesos['pac'] * df_esc['maape_pac']
    )
    df_esc = df_esc.sort_values('maape_esc').reset_index(drop=True)
    df_esc['escenario'] = nombre_esc
    df_esc['estacion'] = nombre_estacion

    # Calcular top 5 y semanas resumen
    top5 = df_esc.head(5)

    #  Imprimir top 5 como validación
    print(f"\n Top 5 años análogos para el escenario '{nombre_esc}':")
    print(top5[['año', 'maape_esc', 'sem_iell']])

    semana_prom = top5['sem_iell'].mean()
    pesos_pond = np.array([0.5, 0.25, 0.1, 0.075, 0.075])
    semana_pond = np.sum(top5['sem_iell'].values * pesos_pond)

    df_final = pd.concat([df_final, pd.DataFrame({
        'año_analizado': [año_actual],
        'mes_analizado': [mes_actual],
        'escenario': [nombre_esc],
        'estacion': [nombre_estacion],
        'semana_promedio': [semana_prom],
        'semana_ponderada': [semana_pond]
    })], ignore_index=True)

# =========================
# RESULTADO FINAL
# =========================

print("\n Resumen de resultados por escenario:")
print(df_final)



 Top 5 años análogos para el escenario 'prec_dom':
    año  maape_esc  sem_iell
0  2002   0.478872        30
1  2004   0.572054        27
2  2001   0.636853        30
3  2009   0.637472        27
4  2015   0.703556        33

 Top 5 años análogos para el escenario 'enso_dom':
    año  maape_esc  sem_iell
0  2015   0.522159        33
1  2004   0.576889        27
2  2003   0.589288        30
3  2005   0.638217        27
4  2007   0.656376        31

 Top 5 años análogos para el escenario 'enso_prec':
    año  maape_esc  sem_iell
0  2015   0.576593        33
1  2004   0.578731        27
2  2003   0.626008        30
3  2002   0.631103        30
4  2007   0.708743        31

 Top 5 años análogos para el escenario 'neutro':
    año  maape_esc  sem_iell
0  2004   0.603946        27
1  2015   0.630836        33
2  2003   0.649115        30
3  2002   0.671058        30
4  2007   0.715385        31

 Top 5 años análogos para el escenario 'atl_dom':
    año  maape_esc  sem_iell
0  2004   0.65011

In [22]:
df_final.año_analizado.value_counts()

año_analizado
2019    5
Name: count, dtype: int64

In [23]:
# =========================
# EVALUACIÓN CONTRA DATO REAL
# =========================

# Obtener semana real del año analizado
semana_real = real.loc[real['year'] == año_actual, 'sem_iell']
if semana_real.empty:
    print(f" No se encontró 'sem_iell' real para el año {año_actual}")
    df_final['sem_iell_real'] = np.nan
    df_final['error_abs_prom'] = np.nan
    df_final['error_abs_pond'] = np.nan
else:
    sem_real = int(semana_real.values[0])
    df_final['sem_iell_real'] = sem_real
    df_final['error_abs_prom'] = (df_final['semana_promedio'].round() - sem_real).abs()
    df_final['error_abs_pond'] = (df_final['semana_ponderada'].round() - sem_real).abs()

df_final


,año_analizado,mes_analizado,escenario,estacion,semana_promedio,semana_ponderada,sem_iell_real,error_abs_prom,error_abs_pond
0,2019,4,prec_dom,puertosanjose,29.4,29.250,25,4.0,4.0
1,2019,4,enso_dom,puertosanjose,29.6,30.600,25,5.0,6.0
2,2019,4,enso_prec,puertosanjose,30.2,30.825,25,5.0,6.0
3,2019,4,neutro,puertosanjose,30.2,29.325,25,5.0,4.0
4,2019,4,atl_dom,puertosanjose,30.2,28.800,25,5.0,4.0


In [ ]:
todos_los_resultados = pd.DataFrame()

for año_actual in range(2023, 2000, -1):  # desde 2020 hasta 2010
    print(f"Procesando año: {año_actual}")

    # Construir series actuales
    serie_actual_prec = construir_serie(prec, año_actual, mes_actual, 'prec')
    serie_actual_atl = construir_serie(atl, año_actual, mes_actual, 'atl')
    serie_actual_pac = construir_serie(pac, año_actual, mes_actual, 'pac')

    if None in [serie_actual_prec, serie_actual_atl, serie_actual_pac]:
        print(f" Datos insuficientes para año {año_actual}, se omite.")
        continue

    resultados = []

    for año_pasado in range(año_actual-1, prec['year'].min()-1, -1):
        serie_ant_prec = construir_serie(prec, año_pasado, mes_actual, 'prec')
        serie_ant_atl = construir_serie(atl, año_pasado, mes_actual, 'atl')
        serie_ant_pac = construir_serie(pac, año_pasado, mes_actual, 'pac')

        if None in [serie_ant_prec, serie_ant_atl, serie_ant_pac]:
            continue

        maape_prec = calcular_maape(serie_actual_prec, serie_ant_prec)
        maape_atl = calcular_maape(serie_actual_atl, serie_ant_atl)
        maape_pac = calcular_maape(serie_actual_pac, serie_ant_pac)

        fila_real = real[real['year'] == año_pasado]
        semana_inicio = int(fila_real['sem_iell'].values[0]) if not fila_real.empty else np.nan

        resultados.append({
            'año': año_pasado,
            'maape_prec': maape_prec,
            'maape_atl': maape_atl,
            'maape_pac': maape_pac,
            'sem_iell': semana_inicio
        })

    if len(resultados) < 8:
        print(f" Menos de 8 años comparables para {año_actual}. Se omite.")
        continue

    df_resultados = pd.DataFrame(resultados)
    df_final = pd.DataFrame()

    for nombre_esc, pesos in escenarios.items():
        df_esc = df_resultados.copy()
        df_esc['maape_esc'] = (
            pesos['prec'] * df_esc['maape_prec'] +
            pesos['atl'] * df_esc['maape_atl'] +
            pesos['pac'] * df_esc['maape_pac']
        )
        df_esc = df_esc.sort_values('maape_esc').reset_index(drop=True)
        df_esc['escenario'] = nombre_esc
        df_esc['estacion'] = nombre_estacion

        top5 = df_esc.head(5)
        print(f"\n Top 5 años análogos para el escenario '{nombre_esc}' en {año_actual}:")
        print(top5[['año', 'maape_esc', 'sem_iell']])

        semana_prom = top5['sem_iell'].mean()
        pesos_pond = np.array([0.5, 0.25, 0.1, 0.075, 0.075])
        semana_pond = np.sum(top5['sem_iell'].values * pesos_pond)

        df_final = pd.concat([df_final, pd.DataFrame({
            'año_analizado': [año_actual],
            'mes_analizado': [mes_actual],
            'escenario': [nombre_esc],
            'estacion': [nombre_estacion],
            'semana_promedio': [semana_prom],
            'semana_ponderada': [semana_pond]
        })], ignore_index=True)

    # Calcular error contra semana real
    semana_real = real.loc[real['year'] == año_actual, 'sem_iell']
    if semana_real.empty:
        print(f" No se encontró semana real para {año_actual}.")
        df_final['sem_iell_real'] = np.nan
        df_final['error_abs_prom'] = np.nan
        df_final['error_abs_pond'] = np.nan
    else:
        sem_real = int(semana_real.values[0])
        df_final['sem_iell_real'] = sem_real
        df_final['error_abs_prom'] = (df_final['semana_promedio'].round() - sem_real).abs()
        df_final['error_abs_pond'] = (df_final['semana_ponderada'].round() - sem_real).abs()

    todos_los_resultados = pd.concat([todos_los_resultados, df_final], ignore_index=True)

# =========================
# RESULTADO TOTAL
# =========================

print("\n Resultado final para todos los años:")
print(todos_los_resultados)

todos_los_resultados.to_excel("data/resultados_analogos_puertosanjose.xlsx", index=False)


Procesando año: 2023
 Datos insuficientes para año 2023, se omite.
Procesando año: 2022
 Datos insuficientes para año 2022, se omite.
Procesando año: 2021
 Datos insuficientes para año 2021, se omite.
Procesando año: 2020

 Top 5 años análogos para el escenario 'prec_dom' en 2020:
    año  maape_esc  sem_iell
0  2004   0.474138        27
1  2019   0.522482        25
2  2005   0.611291        27
3  2015   0.612306        33
4  2002   0.619805        30

 Top 5 años análogos para el escenario 'enso_dom' en 2020:
    año  maape_esc  sem_iell
0  2004   0.407476        27
1  2015   0.482992        33
2  2005   0.489094        27
3  2019   0.585562        25
4  2002   0.672631        30

 Top 5 años análogos para el escenario 'enso_prec' en 2020:
    año  maape_esc  sem_iell
0  2004   0.430357        27
1  2015   0.517977        33
2  2005   0.523534        27
3  2019   0.569681        25
4  2002   0.658934        30

 Top 5 años análogos para el escenario 'neutro' en 2020:
    año  maape_es

In [25]:
todos_los_resultados.año_analizado.max()

np.int64(2020)